In [411]:
import pandas as pd

In [412]:
len("10:30a".split(':')[0])

2

In [413]:
import pandas as pd
import urllib.request
import datetime as dt

def fetch_from_unitime(day_diff, filename="data.csv", start_hour=8, end_hour=20, min_size=0, max_size=500):
    if filename[-4:] != ".csv":
        raise Exception("file must be a csv file")
    if day_diff < 0:
        raise Exception("Day difference must be non-negative")
    if day_diff == 0:
        if start_hour > end_hour:
            raise Exception("Start time must be before end time")
    if start_hour > 23 or end_hour > 23:
        raise Exception("hour must be less than 24")
    if min_size > max_size:
        raise Exception("max_size must be greater than min_size")
    if min_size < 0 or max_size < 0:
        raise Exception("Class size must be non-negative")
    current_time = dt.datetime.today()
    time_delta = dt.datetime.today() + dt.timedelta(days=day_diff)


    link = "https://timetable.mypurdue.purdue.edu/Timetabling/export?output=events.csv&type=room&term=Spring2023PWL&e:from={0:02d}/{1:02d}/{2}&e:to={3:02d}/{4:02d}/{5}&e:after={6}00&e:before={7}00&r:type=genClassroom&r:size={8}..{9}".format(current_time.month, current_time.day, current_time.year, time_delta.month, time_delta.day, time_delta.year, start_hour, end_hour, min_size, max_size)
    urllib.request.urlretrieve(link, '/Users/seanlee/Library/CloudStorage/OneDrive-purdue.edu/2/307/Odyssey/classdata/%s'%filename)

def clean_file(df):
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    for i in range(len(df)):
        start = df.at[i, 'Published Start']
        if start[-1] == 'a' or start[0:2] == '12':
            if len(start.split(':')[0]) == 1:
                df.at[i, 'Published Start'] = '0%s' %(start[:-1])
            else:
                df.at[i, 'Published Start'] = start[:-1]
        elif start == 'noon':
            df.at[i, 'Published Start'] = '12:00'
        else:
            hour = int(start.split(':')[0])
            hour += 12
            rest = start.split(':')[1]
            df.at[i, 'Published Start'] = '%d:%s' %(hour,rest[0:2])


        end = df.at[i, 'Published End']
        if end[-1] == 'a' or end[0:2] == '12':
            if len(end.split(':')[0]) == 1:
                df.at[i, 'Published End'] = '0%s' %(end[:-1])
            else:
                df.at[i, 'Published End'] = end[:-1]
        elif end == 'noon':
            df.at[i, 'Published End'] = '12:00'
        else:
            hour = int(end.split(':')[0])
            hour += 12
            rest = end.split(':')[1]
            df.at[i, 'Published End'] = '%d:%s' %(hour,rest[0:2])

def find_rooms(df):
    building = dict()
    for room in df["Location"].unique():
        if building.get(room.split()[0], 0) == 0:
            building[room.split()[0]] = [room.split()[1]]
        else:
            building[room.split()[0]].append(room.split()[1])
    return building

In [414]:
df = pd.read_csv("week.csv")

In [415]:
clean_file(df)

In [416]:
len(df)

8768

In [417]:
for date in set(df["Date"]):
    for classroom in set(df["Location"]):
        df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '06:00', '6:00', classroom, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]
        df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '22:00', '22:00', classroom, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]

In [418]:
len(df)

11648

In [419]:
len(set(df["Date"]))

5

In [420]:
len(set(df["Location"]))

288

In [421]:
sorted_df = df.sort_values("Published Start")

In [422]:
date_list = []
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    class_list = [[] for i in range(len(sorted_df))]
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        class_list[c_index] = (sorted_df[(sorted_df["Location"] == classroom) & (sorted_df["Date"] == date)])
    date_list.append(class_list)

In [423]:
date_list[0][0]

,Name,Section,Type,Title,Note,Date,Published Start,Published End,Location,Capacity,Instructor / Organization,Email,Requested Services,Approved
11344,<NA>,<NA>,<NA>,<NA>,<NA>,03/06/2023,06:00,6:00,ABE B053,NaN,NaN,NaN,NaN,NaN
445,ASM 21600,19583-002,Lecture,Introduction To Surveying,NaN,03/06/2023,09:30,10:20,ABE B053,32,"Mosier, Nathan Scott (Instr)\nZhang, Wanting (...",mosiern@purdue.edu\nzhan2912@purdue.edu,NaN,09/22/2022
153,ABE 65100N,15495-001,Lecture,Environmental Informatics,NaN,03/06/2023,10:30,11:20,ABE B053,32,"Cherkauer, Keith A (Instr)",cherkaue@purdue.edu,NaN,09/22/2022
128,ABE 32000,65886-001,Lecture,Sld Mod Dyn Sim Anly,NaN,03/06/2023,11:30,12:20,ABE B053,32,"Harlow, Stanley Dean (Instr)",stanley.d.harlow.1@purdue.edu,NaN,09/22/2022
5750,SCLA 10200,25006-249,Lecture,Crit Think & Com II,NaN,03/06/2023,12:30,13:20,ABE B053,32,"Western, David Stanley (Instr)",dswester@purdue.edu,NaN,09/21/2022
5753,SCLA 10200,25007-250,Lecture,Crit Think & Com II,NaN,03/06/2023,13:30,14:20,ABE B053,32,"Western, David Stanley (Instr)",dswester@purdue.edu,NaN,09/21/2022
447,ASM 24500,11119-001,Lecture,Materials Hand & Proc,NaN,03/06/2023,14:30,15:20,ABE B053,32,"Ehlers, Shawn G (Instr)",sehlers@purdue.edu,NaN,09/22/2022
7841,Costa Rica Study Abroad Pre-Trip Meetings,NaN,Special,NaN,NaN,03/06/2023,17:00,19:30,ABE B053,32,NaN,NaN,NaN,10/18/2022
11345,<NA>,<NA>,<NA>,<NA>,<NA>,03/06/2023,22:00,22:00,ABE B053,NaN,NaN,NaN,NaN,NaN
